In [57]:
import json
import pandas as pd
import math
from numpy import diff
from numpy import gradient
import numpy as np
from scipy import pi
from scipy import fft
from scipy.fft import fft, fftfreq
from scipy import signal
import scipy.stats as stats
import plotly.express as px
import plotly.graph_objects as go

from plotly.subplots import make_subplots

In [58]:
def lowpass_filter(column,dict):#column, cutoff, order, sample_rate):
    cutoff = dict['cutoff']
    order = dict['order']  
    rate = dict['rate']
    b,a = signal.butter(order,cutoff/(rate/2),btype='lowpass')
    filtered_channel = signal.filtfilt(b,a,column)
    return (filtered_channel)

def get_psd(df,fs,bin_width):
    
    #fs = len(df)/(df.index[-1]-df.index[0])    
    f, psd = signal.welch(df.to_numpy(), 
                          fs=fs, 
                          nperseg=fs/bin_width,
                          #window='hanning',
                          axis=0
                         )

    df_psd = pd.DataFrame(psd)#,columns=df.columns)
    df_psd.columns
    df_psd['Frequency (Hz)'] = f
    df_psd = df_psd.set_index('Frequency (Hz)')
    
    return df_psd[1:] #drop the first value because it makes the plots look bad and is effectively 0


In [59]:
f=open('OPPOFindX5Pro-03-12-01.json')

In [60]:
data = json.load(f)

In [61]:
print(data.keys())

dict_keys(['MotionIQ', 'rear-streaming-time', 'vehicle-class', 'suspension', 'start-time', 'fork-streamStop', 'speed-wheel-circumference', 'shock-streamStop', 'fork-sag', 'rearReboundLow', 'rear-sensor-type', 'frontModel', 'bike-name', 'shock-present', 'stop-time-str', 'frontReboundLow', 'stop-time', 'location', 'rearSpringRate', 'scalene-pivot-to-eye2', 'publisherUnlocks', 'rearDampLow', 'fork-present', 'front-streaming-time', 'frontPreload', 'fork-worst-rssi', 'sampleRate', 'frontDampLow', 'scalene-enable', 'shock-worst-rssi', 'rearDampHigh', 'rear-axle-travel', 'frontSpringRate', 'speed-wheel-magnets', 'trailConditions', 'frontSpacers', 'fork-max-travel', 'rear-curve-data-version', 'countLocationErrors', 'head-tube-angle', 'recordMode', 'rear-axle-sag', 'scalene-eye-to-eye', 'scalene-pivot-to-eye1', 'elapsed-time', 'frontSpringSetting', 'rearModel', 'fork-disconnects', 'rearTirePressure', 'fork-drops', 'rearSpringSetting', 'time-zone', 'rearSpacers', 'frontTirePressure', 'front-axle

In [62]:
gps_data = data['locations']
frame=pd.DataFrame(gps_data)
print(frame)




             tUTC   latitude     speed    altitude   longitude
0    1.678587e+09 -38.170770  2.440802  635.269043  176.302171
1    1.678587e+09 -38.170732  3.104581  635.488037  176.302190
2    1.678587e+09 -38.170693  3.526130  634.584717  176.302205
3    1.678587e+09 -38.170678  3.729211  634.529907  176.302220
4    1.678587e+09 -38.170647  3.782291  634.529907  176.302231
..            ...        ...       ...         ...         ...
299  1.678587e+09 -38.166481  0.476711  448.283936  176.308617
300  1.678587e+09 -38.166482  0.043975  448.283936  176.308617
301  1.678587e+09 -38.166483  0.002437  447.854187  176.308617
302  1.678587e+09 -38.166483  0.000726  447.693970  176.308618
303  1.678587e+09 -38.166484  0.001362  447.819519  176.308620

[304 rows x 5 columns]


In [63]:
frame['elapsed_time'] = frame['tUTC'] - frame['tUTC'][0]
print(frame)

             tUTC   latitude     speed    altitude   longitude  elapsed_time
0    1.678587e+09 -38.170770  2.440802  635.269043  176.302171         0.000
1    1.678587e+09 -38.170732  3.104581  635.488037  176.302190         1.078
2    1.678587e+09 -38.170693  3.526130  634.584717  176.302205         1.672
3    1.678587e+09 -38.170678  3.729211  634.529907  176.302220         2.270
4    1.678587e+09 -38.170647  3.782291  634.529907  176.302231         2.867
..            ...        ...       ...         ...         ...           ...
299  1.678587e+09 -38.166481  0.476711  448.283936  176.308617       233.126
300  1.678587e+09 -38.166482  0.043975  448.283936  176.308617       233.722
301  1.678587e+09 -38.166483  0.002437  447.854187  176.308617       234.273
302  1.678587e+09 -38.166483  0.000726  447.693970  176.308618       234.798
303  1.678587e+09 -38.166484  0.001362  447.819519  176.308620       235.992

[304 rows x 6 columns]


In [64]:
susp=data['suspension']
df=pd.DataFrame(susp)

In [65]:
df

,time,rearWheel,frontForce,rearForce,shock,fork
0,6159,65.39,-1.56,1.15,25.00,101.01
1,6160,65.34,-1.56,1.15,24.98,100.57
2,6161,65.34,-1.46,0.72,24.98,99.79
3,6162,65.52,-1.46,0.72,25.06,98.66
4,6163,65.84,-1.40,0.52,25.19,97.44
...,...,...,...,...,...,...
47451,53610,9.28,-0.95,0.51,3.27,28.45
47452,53611,9.33,-0.96,0.51,3.29,28.45
47453,53612,9.33,-0.96,0.51,3.29,28.45
47454,53613,9.33,-0.95,0.50,3.29,28.40


In [66]:
size=len(df.index)
timedata,interval=np.linspace(0,size/200,num=size,retstep=True, endpoint=False)
print(timedata,interval)

[0.00000e+00 5.00000e-03 1.00000e-02 ... 2.37265e+02 2.37270e+02
 2.37275e+02] 0.005


In [67]:
df['time']=np.round(timedata,3)

In [68]:
df

,time,rearWheel,frontForce,rearForce,shock,fork
0,0.000,65.39,-1.56,1.15,25.00,101.01
1,0.005,65.34,-1.56,1.15,24.98,100.57
2,0.010,65.34,-1.46,0.72,24.98,99.79
3,0.015,65.52,-1.46,0.72,25.06,98.66
4,0.020,65.84,-1.40,0.52,25.19,97.44
...,...,...,...,...,...,...
47451,237.255,9.28,-0.95,0.51,3.27,28.45
47452,237.260,9.33,-0.96,0.51,3.29,28.45
47453,237.265,9.33,-0.96,0.51,3.29,28.45
47454,237.270,9.33,-0.95,0.50,3.29,28.40


In [69]:
# forkVelocity=np.array(diff(df.fork)/diff(df.time))
# shockVelocity=np.array(diff(df.rearWheel)/diff(df.time))
# v=forkVelocity,shockVelocity
# vd=np.column_stack(v)
# vdata=pd.DataFrame(vd, columns=('forkVelocity','rearVelocity'))
df.where(df.fork>0,0,inplace=True)
df.where(df.shock>0,0,inplace=True)
df.where(df.rearWheel>0,0,inplace=True)
df['forkVelocity']=np.gradient(df.fork,df.time)
df['shockVelocity']=np.gradient(df.shock,df.time)
df['rearVelocity']=np.gradient(df.rearWheel,df.time)

/usr/lib/python3/dist-packages/numpy/lib/function_base.py:1084: RuntimeWarning:

divide by zero encountered in true_divide

/usr/lib/python3/dist-packages/numpy/lib/function_base.py:1084: RuntimeWarning:

invalid value encountered in true_divide

/usr/lib/python3/dist-packages/numpy/lib/function_base.py:1085: RuntimeWarning:

divide by zero encountered in true_divide

/usr/lib/python3/dist-packages/numpy/lib/function_base.py:1085: RuntimeWarning:

invalid value encountered in true_divide

/usr/lib/python3/dist-packages/numpy/lib/function_base.py:1086: RuntimeWarning:

divide by zero encountered in true_divide

/usr/lib/python3/dist-packages/numpy/lib/function_base.py:1086: RuntimeWarning:

invalid value encountered in true_divide

/usr/lib/python3/dist-packages/numpy/lib/function_base.py:1092: RuntimeWarning:

invalid value encountered in multiply

/usr/lib/python3/dist-packages/numpy/lib/function_base.py:1084: RuntimeWarning:

divide by zero encountered in true_divide

/usr/lib/python

In [70]:
#data2=pd.concat([df,vdata], axis=1)
df

,time,rearWheel,frontForce,rearForce,shock,fork,forkVelocity,shockVelocity,rearVelocity
0,0.000,65.39,-1.56,1.15,25.00,101.01,-8.800000e+01,-4.0,-10.0
1,0.005,65.34,-1.56,1.15,24.98,100.57,-1.220000e+02,-2.0,-5.0
2,0.010,65.34,-1.46,0.72,24.98,99.79,-1.910000e+02,8.0,18.0
3,0.015,65.52,-1.46,0.72,25.06,98.66,-2.350000e+02,21.0,50.0
4,0.020,65.84,-1.40,0.52,25.19,97.44,-2.240000e+02,24.0,59.0
...,...,...,...,...,...,...,...,...,...
47451,237.255,9.28,-0.95,0.51,3.27,28.45,0.000000e+00,0.0,0.0
47452,237.260,9.33,-0.96,0.51,3.29,28.45,0.000000e+00,2.0,5.0
47453,237.265,9.33,-0.96,0.51,3.29,28.45,-5.000000e+00,0.0,0.0
47454,237.270,9.33,-0.95,0.50,3.29,28.40,5.684342e-11,-2.0,-5.0


In [71]:

df.describe()

,time,rearWheel,frontForce,rearForce,shock,fork,forkVelocity,shockVelocity,rearVelocity
count,47456.000000,47456.000000,47456.000000,47456.000000,47456.000000,47456.000000,47436.000000,47436.000000,47436.000000
mean,118.588065,41.025055,-1.037335,0.386361,15.365579,62.496756,-0.265305,-0.109031,-0.280377
std,68.525680,18.954279,2.260531,1.732744,7.491857,22.711307,474.193059,135.098837,341.469588
min,0.000000,0.000000,-16.000000,-10.220000,0.000000,0.000000,-1904.000000,-445.000000,-1018.000000
25%,59.258750,28.670000,-2.000000,-0.450000,10.440000,45.240000,-210.000000,-58.000000,-147.000000
50%,118.597500,39.730000,-0.940000,0.340000,14.700000,60.530000,-10.000000,-7.000000,-18.000000
75%,177.936250,51.890000,0.212500,1.020000,19.520000,78.060000,83.000000,11.000000,28.000000
max,237.275000,129.750000,10.440000,15.870000,52.610000,161.210000,4487.000000,2225.000000,5377.000000


In [72]:
df.idxmax()

time             47455
rearWheel        39723
frontForce        6480
rearForce         4908
shock            39723
fork             39727
forkVelocity     42204
shockVelocity    13993
rearVelocity     13993
dtype: int64

In [98]:
dict = {
    'cutoff' : 20,
    'order' : 10,
    'rate' : 200
}
#fig = make_subplots(specs=[[{"secondary_y": True}]])
fig=go.Figure()
#fig=px.scatter(x=df.rearWheel, y=df.rearVelocity, text=df.time)
fig.add_trace(go.Scatter(x=df.time, y=df.forkVelocity))
fig.add_trace(go.Scatter(x=frame.elapsed_time, y=frame.speed, yaxis="y1"))
fig.update_layout(
#     # title=f"Frequency analysis ",
#     # xaxis_title="Frequency (Hz)",
#     # yaxis_title="Acceleration (g^2/Hz)",
    width=800, height=600
    )
# fig.update_layout(
#     yaxis1=dict(
#         title="Position",
#         anchor="free",
#         overlaying="y",
#         side="right",
#         position=0.05
#     ))
fig.update_traces(
    marker_size=2
    ) 
fig.show()

In [74]:
start=int(40 * 200)
stop=int(48 * 200)
sample= df.fork[start:stop]
time=df.time[start:stop]
power=get_psd(sample,200,0.1)
print(sample)

8000     95.24
8001     93.93
8002     92.41
8003     90.56
8004     88.46
         ...  
9595     89.53
9596     94.41
9597     98.76
9598    103.15
9599    107.65
Name: fork, Length: 1600, dtype: float64


/home/jono/.local/lib/python3.10/site-packages/scipy/signal/_spectral_py.py:2014: UserWarning:

nperseg = 2000 is greater than input length  = 1600, using nperseg = 1600



In [75]:
print(power)

                           0
Frequency (Hz)              
1.000           1.754928e+01
1.125           7.226065e+01
1.250           2.051564e+02
1.375           9.849275e+01
1.500           1.091270e+02
...                      ...
99.500          4.062511e-05
99.625          2.518325e-05
99.750          1.334119e-06
99.875          2.639321e-06
100.000         4.982560e-07

[793 rows x 1 columns]


In [76]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Scatter(x=power.index[1:], y=power[0]),
              )
                        
            

fig.update_layout(
    title=f"Frequency analysis ",
    xaxis_title="Frequency (Hz)",
    yaxis_title="Movement",
    width=1000, height=600) 
#fig.update_traces(marker_line_width=0.01, selector=dict(type='scatter'))                
fig.update_xaxes(type="log")
#fig.update_yaxes(type="log")

fig.show()